In [1]:
using CairoMakie
using CSV
using DataFrames
using MLJ
using MLJBase

In [21]:
# Import and unpack training dataset function
function get_data(csv_path, col_names, new_col_names)
    df = CSV.read(csv_path, DataFrame, missingstring="NA") |>
    x -> DataFrames.select(x, col_names) |>
    x -> rename(x, new_col_names) |>
    dropmissing |>
    x -> coerce(x, :pl=>Continuous, :sx=>Continuous, :snr=>Multiclass, :smr=>Multiclass)
    # Split predictors/responde variable
    y, X = unpack(df, ==(:biomass), _->true; rng=1010)
    return y, X
end


get_data (generic function with 3 methods)

In [22]:
csv_path = "C:/Users/julio/Downloads/lichen_training.csv"
col_names = ["Total.lichen", "basal_area", "PL", "mean_Long", "mean_Lat", "Aspect_south", "SNR", "SMR", "SX"]
new_col_names = ["biomass", "basalarea", "pl", "lon", "lat", "south", "snr", "smr", "sx"]

y, X = get_data(csv_path, col_names, new_col_names)

([2.43, 0.34, 1.72, 288.4, 1.49, 1917.77, 254.57, 3044.13, 1.55, 67.36  …  41.94, 3209.72, 294.86, 1.93, 794.04, 2.99, 523.83, 50.19, 1164.63, 21.75], 77×8 DataFrame
 Row │ basalarea  pl       lon       lat      south    snr   smr   sx      
     │ Float64    Float64  Float64   Float64  Float64  Cat…  Cat…  Float64 
─────┼─────────────────────────────────────────────────────────────────────
   1 │     31.17     78.0  -124.62   56.2372     1.0   C     4         0.0
   2 │     30.08     50.0  -125.133  56.0107     0.55  C     3         0.0
   3 │     22.54      0.0  -124.625  55.4193     0.39  C     4        60.0
   4 │     11.71     79.0  -125.446  56.0079     0.59  B     3         0.0
   5 │     24.79     80.0  -125.487  55.7387     0.67  C     4         0.0
   6 │     19.94    100.0  -124.691  55.6586     0.78  B     4         0.0
   7 │     27.86      0.0  -125.83   55.807      0.96  B     4         0.0
   8 │      2.49    100.0  -124.244  55.5477     0.43  C     3         0.0
  ⋮  │

In [27]:
# Create a train a model
function model_data(X, y)
    # Load model and create pipeline
    model = @load RandomForestRegressor pkg=DecisionTree verbosity=0
    pipe = @pipeline Standardizer OneHotEncoder model target=v->log.(v) inverse=v->exp.(v)
    # Create a machine
    pipe.random_forest_regressor.n_trees = 50
    m = machine(pipe, X, y)

    return m
end

LoadError: LoadError: UndefVarError: model not defined
in expression starting at c:\Users\julio\Desktop\jl-sandbox\code\lichen-functions-linear-randomforest.ipynb:5

In [ ]:
# Split predictors (X) and response variable (y)
y, X = unpack(lichen_training, ==(:biomass_log), _ -> true);

In [ ]:
# Load a linear regressor and a random forest regressor
LinearRegressor = @load LinearRegressor pkg=MLJLinearModels
RFRegressor = @load RandomForestRegressor pkg=DecisionTree

In [ ]:
# Instantiate a pipeline
RegressorPipe = @pipeline(Standardizer(), LinearRegressor(), RFRegressor())